<a   href="https://colab.research.google.com/github/N-Nieto/Relevance_Based_Pruning/Relevance_based_pruning_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# This is a toy example for the implementation of Relevance Pruning Method 

### Import GitHub Repository

In [1]:
!git clone https://github.com/N-Nieto/Relevance_Based_Pruning  --quiet

### Imports

In [2]:
# Import libraries
import timeit
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from Relevance_Based_Pruning.Python_Implementation import RBP_Utilitys

Load Syntetic data

In [3]:
# Change to your data direction
file_name= '/content/Relevance_Based_Pruning/Python_Implementation/Synthetic_data.mat'

# In[]: Data Load
X_load= sio.loadmat(file_name=file_name)
X_data= X_load['X']
Y_data= X_load['Y']
   
del (X_load , file_name)


Hyperparameter setting

In [4]:
# Create an RBP object with all the necessary functions inside
rbp = RBP_Utilitys.RBP()

# In[]: Parameters
test_size = 0.9

N_nodes_max = 500

# Number of diferent random initializations
K = 5

# Cross Validations splits
CV = 2

Main Loop

In [ ]:
# In[]
for cv in range (CV):
    x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=test_size, shuffle=True)
    
    for k in range (K):
        acc_prun = 0
        

        start_time = timeit.default_timer()
        # Generate a random network
        W , b = rbp.generate_rand_network(x_train, N_nodes_max)
        
        # Fitting with pinv2
        B = rbp.fit(x=x_train,W=W,b=b,y=y_train)
                
        # Pruning: the pruning step could be changed for faster implementations
        for N_nodes_to_keep in range(N_nodes_max):
            
            if not(N_nodes_to_keep == 0):
                    
                # Prune the network
                W_prun , b_prun, B_prun = rbp.fix_prunning(W, b, B, prn_perc = N_nodes_to_keep, mode = "keep")
                
                # Make a prediction with the pruned netwokr
                y_pred_tst = rbp.predict(x_test, W_prun, b_prun, B_prun)
                
                # Calculaty the accuracy
                acc_aux=accuracy_score(y_test,y_pred_tst)
                
                # Append the accuracy for network size
                acc_prun=np.append(acc_prun,acc_aux)
            
        stop_time = timeit.default_timer()
        #compute 
        time = stop_time - start_time

        acc_prun=np.delete(acc_prun,0)
        
        if k == 0:    
            #Inicialiced
            Acc_prun=acc_prun
            train_time = time
        else:
            #Stack trials
            Acc_prun = np.vstack((Acc_prun,acc_prun))
            train_time = np.append(train_time,time)
    
    if cv == 0:    
        #Inicialiced
        Acc_final=Acc_prun
        final_time = train_time
    else:
        #Stack trials
        Acc_final = np.vstack((Acc_final,Acc_prun))
        final_time = np.append(final_time,train_time)
    


Ploting Results

In [ ]:
## Ploting
print("Mean time for each network training: "final_time.mean())
plt.figure(figsize= [20,10])
plt.plot(np.mean(Acc_final*100,0))
plt.title("RBP performance over syntecic data")
plt.xlabel("Hidden Nodes")
plt.ylabel("Accuracy [%]")